# **SpaceX Falcon 9 First Stage Landing Predition**
## EDA with SQL

### Objectives

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


#### Download the datasets

Download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


#### Connect to the database

Load the SQL extension & Establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Write and Execute SQL queries for the following:



#### Display the names of the unique launch sites  in the space mission


In [7]:
%sql SELECT DISTINCT Launch_Site from SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40




####  Display 5 records where launch sites begin with the string 'CCA' 


In [8]:
%sql SELECT * from SPACEXTBL where Launch_Site like 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt




#### Display the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%sql SELECT SUM(Payload_Mass__KG_) as Total_PL from SPACEXTBL where Customer like 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Total_PL
45596




#### Display average payload mass carried by booster version F9 v1.1


In [10]:
%sql SELECT AVG(Payload_Mass__KG_) as Avg_PL from SPACEXTBL where Booster_Version like 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Avg_PL
2928.4




#### List the date when the first succesful landing outcome in ground pad was acheived


In [11]:
%sql SELECT min(Date) as First_Success from SPACEXTBL where "Landing _Outcome" = "Success (ground pad)";

 * sqlite:///my_data1.db
Done.


First_Success
01-05-2017




#### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [12]:
%sql SELECT Booster_Version, Payload_Mass__KG_, "Landing _Outcome" from SPACEXTBL where "Landing _Outcome" = 'Success (drone ship)' and Payload_Mass__KG_ between 4000 and 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_,Landing _Outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)






#### List the total number of successful and failure mission outcomes


In [13]:
%sql SELECT Mission_Outcome, Count(*) as Total from SPACEXTBL group by Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1





#### List the   names of the booster_versions which have carried the maximum payload mass


In [14]:
%sql SELECT Booster_Version from SPACEXTBL where Payload_Mass__KG_ = (SELECT MAX(Payload_Mass__KG_) from SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6





#### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

##### **Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [15]:
%sql SELECT substr(Date, 4, 2) as Month, substr(Date, 7, 4) as Year, "Landing _Outcome", Booster_Version, Launch_Site from SPACEXTBL where "Landing _Outcome" like 'Failure (drone ship)' and Year='2015';

 * sqlite:///my_data1.db
Done.


Month,Year,Landing _Outcome,Booster_Version,Launch_Site
01,2015,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,2015,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40





#### Rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order


In [16]:
%sql SELECT "Landing _Outcome", Count(*) as Total_Sc from SPACEXTBL where "Landing _Outcome" like 'Success%' and Date between '04-06-2010' and '20-03-2017' group by "Landing _Outcome" order by Total_Sc DESC;

 * sqlite:///my_data1.db
Done.


Landing _Outcome,Total_Sc
Success,20
Success (drone ship),8
Success (ground pad),6
